<a href="https://colab.research.google.com/github/sash-ko/ml_playgound/blob/master/social_distancing/Social-distancing-tool.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[Social distancing detection tool](https://www.analyticsvidhya.com/blog/2020/05/social-distancing-detection-tool-deep-learning/)

Works only on [Google colab](https://colab.research.google.com/) because it requires GPUs

In [0]:
%matplotlib inline
import matplotlib.pyplot as plt

import numpy as np
import seaborn as sns

from pylab import rcParams
rcParams['figure.figsize'] = (10, 7)

import pandas as pd
pd.set_option('max_columns', 40)
pd.set_option('max_rows', 20)
pd.set_option('precision', 3)

In [0]:
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

import cv2
import random

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog

In [0]:
% % time

!rm - r frames/*
!mkdir frames/

# specify path to video
video = "sample.mp4"

# capture video
cap = cv2.VideoCapture(video)
cnt = 0

# Check if video file is opened successfully
if cap.isOpened() == False:
    print("Error opening video stream or file")

ret, first_frame = cap.read()

# Read until video is completed
while(cap.isOpened()):

    # Capture frame-by-frame
    ret, frame = cap.read()

    if ret:

        # save each frame to folder
        cv2.imwrite(f'frames/{cnt}.png', frame)
        cnt = cnt + 1

        if(cnt == 750):
            break

    # Break the loop
    else:
        break

rm: frames/*: No such file or directory
CPU times: user 51 s, sys: 5.59 s, total: 56.6 s
Wall time: 57.8 s


In [0]:
#frame rate of a video
FPS = cap.get(cv2.CAP_PROP_FPS)

print(FPS)

25.0


In [0]:
cfg = get_cfg()

# add project-specific config (e.g., TensorMask) here if you're not 
# running a model in detectron2's core library
cfg.merge_from_file(model_zoo.get_config_file(
    "COCO-Detection/faster_rcnn_R_50_C4_3x.yaml"))

# set threshold for this model
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.9  

# Find a model from detectron2's model zoo. You can use the https://dl.fbaipublicfiles... url as well
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(
    "COCO-Detection/faster_rcnn_R_50_C4_3x.yaml")

predictor = DefaultPredictor(cfg)

AssertionError: Torch not compiled with CUDA enabled

In [0]:
#read an image
img = cv2.imread("frames/30.png")

#pass to the model
outputs = predictor(img)

In [0]:
# Use `Visualizer` to draw the predictions on the image.
v = Visualizer(img[:, :, ::-1],
               MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)

v = v.draw_instance_predictions(outputs["instances"].to("cpu"))

cv2_imshow(v.get_image()[:, :, ::-1])

In [0]:
classes = outputs['instances'].pred_classes.cpu().numpy()
print(classes)

bbox = outputs['instances'].pred_boxes.tensor.cpu().numpy()
print(bbox)

In [0]:
#identity only persons 
ind = np.where(classes == 0)[0]

#identify bounding box of only persons
person = bbox[ind]

#total no. of persons
num = len(person)

x1, y1, x2, y2 = person[0]
print(x1, y1, x2, y2)

In [0]:
img = cv2.imread('frames/30.png')
_ = cv2.rectangle(img, (x1, y1), (x2, y2), (255,0,0), 2)

plt.figure(figsize=(20,10))
plt.imshow(img)

In [0]:
#compute center 
x_center = int((x1 + x2) / 2)
y_center = int(y2)

center = (x_center, y_center)

_ = cv2.circle(img, center, 5, (255, 0, 0), -1)
plt.figure(figsize=(20, 10))
plt.imshow(img)

In [0]:
def mid_point(img, person, idx):
    """Function which return the bottom center of every bbox"""

    # get the coordinates
    x1, y1, x2, y2 = person[idx]
    _ = cv2.rectangle(img, (x1, y1), (x2, y2), (0, 0, 255), 2)

    # compute bottom center of bbox
    x_mid = int((x1 + x2)/2)
    y_mid = int(y2)
    mid = (x_mid, y_mid)

    _ = cv2.circle(img, mid, 5, (0, 0, 255), -1)
    cv2.putText(img, str(idx), mid, cv2.FONT_HERSHEY_SIMPLEX,
                1, (255, 255, 255), 2, cv2.LINE_AA)

    return mid

midpoints = [mid_point(img, person, i) for i in range(len(person))]

# visualize image
plt.figure(figsize=(20, 10))
plt.imshow(img)

In [0]:
from scipy.spatial import distance

def compute_distance(midpoints, num):
    dist = np.zeros((num, num))
    
    for i in range(num):
        for j in range(i+1, num):
            if i != j:
                dst = distance.euclidean(midpoints[i], midpoints[j])
                dist[i][j] = dst
    return dist

def find_closest(dist, num, thresh):
    p1 = []
    p2 = []
    d = []
    for i in range(num):
        for j in range(i, num):
            if((i != j) & (dist[i][j] <= thresh)):
                p1.append(i)
                p2.append(j)
                d.append(dist[i][j])
                
    return p1, p2, d

In [0]:
thresh = 100

dist = compute_distance(midpoints, num)

p1, p2, d = find_closest(dist, num, thresh)
df = pd.DataFrame({"p1": p1, "p2": p2, "dist": d})
df

In [0]:
def change_2_red(img, person, p1, p2):
    risky = np.unique(p1+p2)
    for i in risky:
        x1, y1, x2, y2 = person[i]
        _ = cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 0), 2)
    return img


img = change_2_red(img, person, p1, p2)

plt.figure(figsize=(20, 10))
plt.imshow(img)

In [0]:
import os
import re

names = os.listdir('frames/')
names.sort(key=lambda f: int(re.sub('\D', '', f)))

In [0]:
def find_closest_people(name, thresh):
    img = cv2.imread(f'frames/{name}')
    outputs = predictor(img)
    classes = outputs['instances'].pred_classes.cpu().numpy()
    bbox = outputs['instances'].pred_boxes.tensor.cpu().numpy()
    ind = np.where(classes == 0)[0]
    person = bbox[ind]
    midpoints = [mid_point(img, person, i) for i in range(len(person))]
    num = len(midpoints)
    
    dist = compute_distance(midpoints, num)
    p1, p2, d = find_closest(dist, num, thresh)
    
    img = change_2_red(img, person, p1, p2)
    cv2.imwrite(f'frames/{name}', img)
    
    return 0

In [0]:
from tqdm import tqdm

thresh = 100
_ = [find_closest_people(names[i], thresh) for i in tqdm(range(len(names)))]

In [0]:
%%time

frames = os.listdir('frames/')
frames.sort(key=lambda f: int(re.sub('\D', '', f)))

frame_array = []

for i in range(len(frames)):

    # reading each files
    img = cv2.imread(f'frames/{frames[i]}')
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    height, width, layers = img.shape
    size = (width, height)

    # inserting the frames into an image array
    frame_array.append(img)

out = cv2.VideoWriter('sample_output.mp4',
                      cv2.VideoWriter_fourcc(*'DIVX'), 25, size)

for i in range(len(frame_array)):
    # writing to a image array
    out.write(frame_array[i])
out.release()